In [1]:
import numpy as np
import pandas as pd
import utils_jie
import sys
sys.path.insert(1, './run_FACT')
from dataset import data_preprocess
from path_search import SearchPathPermutation
from path_shap2 import CalPathContribution, TrainFeaturePredictorsResidual
import networkx as nx
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import os.path as osp
import os
import copy
import random
import xgboost
import json
data_rnd_seed = 42
use_inactive = 1
model_rnd_seed = 42
begin_idx = 1
dataset = 'uti'

In [2]:
# Initialization
A_name = 'Race_bin'
y_label = 'Recurrence'
Graph_idx = 2
model_class = 'xgboost'

out_dir = "./results/"

#df_tmp = pd.read_csv('hivdataforfacts.csv', index_col=0)
f_names = ['Race_bin','Sex','UTI_Age','CI_score','Area','Median_household_income','Crime_rate','Medical_rate','Uninsured_perc']

In [3]:
# read data binary
#df_ex = pd.read_csv('hivdataforfacts.csv', usecols=['HIV_diagnosis','city','gender','msm','race','ethnicity','age','drug_abuse','income','education','health_insurance','ruralityNum','healthcare_facilities','drinking','violent_crime','smoking','rurality'])
#df_ex.rename(columns = {'edu':'EDUCATION', 'employment2':'EMPLOYMENT', 'financial2':'FINANCIAL', 'livingcon':'LIVING_CON', 
#                        'livingsuply':'LIVING_SUPLY', 'insurance2':'insurance_type'}, inplace = True)

# read data three level #
df_ex = pd.read_csv('data_combined_only.csv',
                    usecols=['PID','Recurrence','Race_bin','Sex','UTI_Age','CI_score','Area','Median_household_income','Crime_rate','Medical_rate','Uninsured_perc'])
#df_ex.rename(columns = {'edu3':'EDUCATION', 'employment3':'EMPLOYMENT', 'financial3':'FINANCIAL', 'livingcon3':'LIVING_CON', 
#                      'livingsuply3':'LIVING_SUPLY', 'insurance3':'insurance_type'}, inplace = True)
df_ex.head(5)
display(df_ex.dtypes)
df_ex = df_ex.astype({"Sex":'int'}) 

PID                          int64
Recurrence                   int64
Sex                          int64
Race_bin                     int64
UTI_Age                      int64
CI_score                     int64
Area                         int64
Median_household_income    float64
Crime_rate                 float64
Medical_rate               float64
Uninsured_perc             float64
dtype: object

In [4]:
# analyze data
#df_ex = pd.merge(df_tmp[['PATID', 'race_ethnicity', 'HbA1c', 'Outcome', 'ndi', y_label]], df_ex)
#print(df_ex.groupby('race_ethnicity')['race_ethnicity'].agg('count'))
#print(df_ex.groupby('EDUCATION')['EDUCATION'].agg('count'))
#print(df_ex.groupby('EMPLOYMENT')['EMPLOYMENT'].agg('count'))
#print(df_ex.groupby('FINANCIAL')['FINANCIAL'].agg('count'))
#print(df_ex.groupby('LIVING_CON')['LIVING_CON'].agg('count'))
#print(df_ex.groupby('LIVING_SUPLY')['LIVING_SUPLY'].agg('count'))
#print(df_ex.groupby('insurance_type')['insurance_type'].agg('count'))
#print(df_ex.groupby('Outcome')['Outcome'].agg('count'))
#df_ex['race_ethnicity'] = df_ex['race_ethnicity'][df_ex['race_ethnicity'] == 'NHB']
#df_ex

In [5]:
# read edge data
edge_list = utils_jie.load_graph_edges('utigraphforfacts_recurrence_full_wo_outcome.csv')
edge_list

[('Area', 'Medical_rate', {'From': 'Area'}),
 ('Area', 'Crime_rate', {'From': 'Area'}),
 ('Area', 'Income', {'From': None}),
 ('Medical_rate', 'CI_score', {'From': 'Medical_rate'}),
 ('Race_bin', 'CI_score', {'From': 'Race_bin'}),
 ('Race_bin', 'Crime_rate', {'From': 'Race_bin'}),
 ('Race_bin', 'Median_household_income', {'From': 'Race_bin'}),
 ('Race_bin', 'Uninsured_perc', {'From': 'Race_bin'}),
 ('CI_score', 'Uninsured_perc', {'From': None}),
 ('Median_household_income', 'CI_score', {'From': 'Median_household_income'}),
 ('Median_household_income',
  'Uninsured_perc',
  {'From': 'Median_household_income'}),
 ('Uninsured_perc', 'Medical_rate', {'From': 'Uninsured_perc'}),
 ('Sex', 'Median_household_income', {'From': 'Sex'}),
 ('Sex', 'CI_score', {'From': 'Sex'}),
 ('Sex', 'Uninsured_perc', {'From': 'Sex'}),
 ('UTI_Age', 'CI_score', {'From': 'UTI_Age'}),
 ('UTI_Age', 'Uninsured_perc', {'From': 'UTI_Age'})]

In [6]:
df_ex.head(5)

,PID,Recurrence,Sex,Race_bin,UTI_Age,CI_score,Area,Median_household_income,Crime_rate,Medical_rate,Uninsured_perc
0,28,1,1,1,77,23,0,40130.5,0.32,189.30,25.6
1,45,0,1,0,55,5,0,40792.9,0.36,186.84,25.1
2,48,0,1,1,67,6,0,42550.0,0.35,200.09,21.9
3,61,0,1,1,75,9,0,40412.2,0.30,185.82,27.3
4,65,0,1,1,86,9,0,40412.2,0.30,185.82,27.3


In [7]:
y_label

'Recurrence'

In [8]:
# read edge data
edge_list = utils_jie.load_graph_edges('utigraphforfacts_recurrence_full_wo_outcome.csv')
data_dict, f_types, f_map = \
    utils_jie.preprocessing_data(df_ex, f_names, y_label, A_name, 0.3, 42)

X_train, X_train_ori, y_train, a_train = \
    data_dict['X_train'], data_dict['X_train_ori'], data_dict['y_train'], data_dict['a_train']
X_test, X_test_ori, y_test, a_test = \
    data_dict['X_test'], data_dict['X_test_ori'], data_dict['y_test'], data_dict['a_test']

In [9]:
def path_transfer(path_dict):
    new_path_dict = {}
    for path in path_dict:
        new_path = "-".join(list(path))
        new_path_dict[new_path] = path_dict[path]

    return new_path_dict

In [10]:
G = nx.Graph()
G.add_nodes_from(f_names)
G.add_edges_from(edge_list)
active_nodes, dir_pre_dict, node_permutations, edge_permutations, path_permutations = SearchPathPermutation(G, A_name)
graph_info = {"f_map": f_map, "active_nodes": active_nodes, "dir_pre_dict": dir_pre_dict, "A_name": A_name}

In [11]:
predictors, X_train_res, X_test_res, active_idx, inactive_idx = TrainFeaturePredictorsResidual(graph_info, X_train, X_test, f_types)
graph_info["active_idx"], graph_info["inactive_idx"] = active_idx, inactive_idx

In [12]:
if model_class == "mlp":
    model = MLPClassifier(hidden_layer_sizes=(8,), random_state=model_rnd_seed, warm_start=True)
    model.fit(X_train[:, begin_idx:], y_train)
    pred_train = model.predict(X_train[:, begin_idx:])
    pred_test = model.predict(X_test[:, begin_idx:])
elif model_class == "xgboost":
    model = xgboost.train({"learning_rate": 0.01}, xgboost.DMatrix(X_train[:, begin_idx:], label=y_train), 1000)
    pred_train = (model.predict(xgboost.DMatrix(X_train[:, begin_idx:])) > 0.5)
    pred_test = (model.predict(xgboost.DMatrix(X_test[:, begin_idx:])) > 0.5)
else:
    model = LogisticRegression(random_state=model_rnd_seed)
    model.fit(X_train[:, begin_idx:], y_train)
    pred_train = model.predict(X_train[:, begin_idx:])
    pred_test = model.predict(X_test[:, begin_idx:])

In [13]:
DP_train = pred_train[a_train == 1].mean() - pred_train[a_train == 0].mean()
DP_test = pred_test[a_test == 1].mean() - pred_test[a_test == 0].mean()
print("Total GT DP", y_test[a_test == 1].mean() - y_test[a_test == 0].mean())
print("Total DP", DP_train, DP_test)

print((pred_test[y_test == 0])[a_test[y_test == 0] == 1].mean(),
      (pred_test[y_test == 0])[a_test[y_test == 0] == 0].mean())
print((pred_test[y_test == 1])[a_test[y_test == 1] == 1].mean(),
      (pred_test[y_test == 1])[a_test[y_test == 1] == 0].mean())
print("Total EO", DP_train, DP_test)
Acc_train = (pred_train == y_train).mean()
Acc_test = (pred_test == y_test).mean()
print("Train Accuracy", Acc_train)
print("Test Accuracy", Acc_test)
print("Number of permutation", len(path_permutations))
path_ctb_dp_train = {tuple(node): [] for node in path_permutations[0]}
path_ctb_acc_train = {tuple(node): [] for node in path_permutations[0]}
path_ctb_dp_test = {tuple(node): [] for node in path_permutations[0]}
path_ctb_acc_test = {tuple(node): [] for node in path_permutations[0]}

Total GT DP -0.012067299584152896
Total DP -0.0023478559323851823 -0.005000787342147216
0.007942811755361398 0.008802816901408451
0.007352941176470588 0.03007518796992481
Total EO -0.0023478559323851823 -0.005000787342147216
Train Accuracy 0.8398002305032655
Test Accuracy 0.8145161290322581
Number of permutation 3


In [14]:
rnd_idxes = [idx for idx in range(len(path_permutations))]
random.shuffle(rnd_idxes)
s_size = min(5, len(path_permutations))
for rnd_idx in rnd_idxes[:s_size]:
    path_ctb_dp_train, path_ctb_acc_train = CalPathContribution(model, predictors, graph_info, X_train, a_train,
                                                                y_train, X_train_res, path_permutations[rnd_idx],
                                                                node_permutations[rnd_idx], path_ctb_dp_train,
                                                                path_ctb_acc_train)
    path_ctb_dp_test, path_ctb_acc_test = CalPathContribution(model, predictors, graph_info, X_test, a_test,
                                                              y_test, X_test_res, path_permutations[rnd_idx],
                                                              node_permutations[rnd_idx], path_ctb_dp_test,
                                                              path_ctb_acc_test)

print("Path#Contribution to Disparity#Contribution to Accuracy")
for path_key in path_ctb_acc_test:
    if len(path_ctb_acc_test[path_key]) > 0 and "R#" not in path_key[0]:
        path_ctb_dp_train[path_key] = np.array(path_ctb_dp_train[path_key]).mean()
        path_ctb_acc_train[path_key] = np.array(path_ctb_acc_train[path_key]).mean()
        path_ctb_dp_test[path_key] = np.array(path_ctb_dp_test[path_key]).mean()
        path_ctb_acc_test[path_key] = np.array(path_ctb_acc_test[path_key]).mean()
        print(path_key, "#", path_ctb_dp_test[path_key], "#", path_ctb_acc_test[path_key])

Path#Contribution to Disparity#Contribution to Accuracy
('Race_bin', 'Crime_rate') # -0.00021772262138036158 # 0.00014934289127836267
('Race_bin', 'Median_household_income') # 0.001741780971042891 # 0.0005973715651134507
('Race_bin', 'CI_score:Medical_rate:Uninsured_perc') # -0.002177226213803614 # -0.00029868578255672534
('Race_bin', 'Median_household_income', 'CI_score:Medical_rate:Uninsured_perc') # 0.0006531678641410847 # -0.0013440860215053752


In [15]:
out_fname = osp.join(out_dir, f"FACTS_{model_class}_Output_{y_label}_Graph{Graph_idx}_recurrence_full_1202.json")

if not osp.exists(out_dir):
    os.makedirs(out_dir)
with open(out_fname, "w") as out_f:
    out_str = json.dumps(
        {'path_ctb_dp_train': path_transfer(path_ctb_dp_train), 'path_ctb_acc_train': path_transfer(path_ctb_acc_train),
         'path_ctb_dp_test': path_transfer(path_ctb_dp_test), 'path_ctb_acc_test': path_transfer(path_ctb_acc_test),
         'DP_train': DP_train, 'DP_test': DP_test, 'Acc_train': Acc_train, 'Acc_test': Acc_test})
    out_f.write(out_str)